In [12]:
import os
import json
import numpy as np

In [13]:
path_or_dict = 'configs/smpl.json'
os.path.isfile(path_or_dict)
with open(path_or_dict, "r") as f:
                json_dict = json.load(f)
json_dict               

{'experiment': {'epochs': 5000,
  'batch_size': 32,
  'learning_rate': 0.001,
  'temporal_window_size': 0.5,
  'reflect_probability': 0.0,
  'motion_augmentation': 0.0,
  'checkpoint': 'smpl'},
 'body': {'model': 'smpl_female_neutral',
  'skeleton': 'smpl',
  'input_joints': [0, 1, 2, 3, 4, 5, 6, 9, 12, 13, 14, 16, 17, 18, 19]},
 'garment': {'name': 'tshirt', 'blend_weights_smoothing_iterations': 10},
 'data': {'dataset': 'smpl',
  'train': 'toy.txt',
  'validation': 'toy.txt',
  'test': 'toy.txt',
  'fps': 15},
 'model': {'blend_weights_optimize': False},
 'loss': {'cloth': {'type': 'baraff98', 'stretch': 10.0, 'shear': 1.0},
  'bending': 2e-05,
  'collision_weight': 10.0,
  'collision_threshold': 0.004,
  'density': 0.15,
  'pinning': 100.0,
  'gravity': '-Y'}}

In [14]:
def triangulate(faces):
    triangles = np.int32(
        [triangle for polygon in faces for triangle in _triangulate_recursive(polygon)])
    ''' 
        triangles = []
        for polygon in faces:
            for triangle in _triangulate_recursive(polygon):
                triangles.append(triangle)
    '''
        
    
    return triangles


def _triangulate_recursive(face): # 处理polygon
    if len(face) == 3:
        return [face]
    else:
        return [face[:3]] + _triangulate_recursive([face[0], *face[2:]]) # *表示解包元素并合并,
                                                                         #等效于facep[0]+face[2:]

In [15]:

body_model = '../body_models/smpl_female_neutral/body.npz'
with np.load(body_model) as model:
    vertices = model["vertices"]
    faces = triangulate(model["faces"])
    blend_weights = model["blend_weights"].astype(np.float32)
    joints = np.float32(model["joints"])
    rest_pose = model["rest_pose"]
    parents = model["parents"]


In [16]:
polygon_test = [2,4,5,6,78,10]
_triangulate_recursive(polygon_test)

[[2, 4, 5], [2, 5, 6], [2, 6, 78], [2, 78, 10]]

In [17]:
parents

array([-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13,
       14, 16, 17, 18, 19, 20, 21])

In [18]:
joints

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.0713612 , -0.08958381, -0.00804619],
       [-0.06901202, -0.08896044, -0.00479569],
       [-0.00250821,  0.10325686, -0.02218514],
       [ 0.10203011, -0.45379317, -0.0147351 ],
       [-0.10516441, -0.4596109 , -0.01398101],
       [ 0.00107275,  0.2309145 , -0.02389815],
       [ 0.09047622, -0.8371416 , -0.05823671],
       [-0.09113577, -0.8432487 , -0.05587261],
       [ 0.00309993,  0.27998674,  0.00396906],
       [ 0.11381389, -0.89178646,  0.05613341],
       [-0.11317899, -0.8896583 ,  0.0620271 ],
       [ 0.0011373 ,  0.48822993, -0.04579642],
       [ 0.07839821,  0.39776716, -0.03290576],
       [-0.07393289,  0.39559293, -0.0378416 ],
       [ 0.00465446,  0.5505521 ,  0.00440884],
       [ 0.16371532,  0.42950618, -0.0401986 ],
       [-0.16313575,  0.42837778, -0.0476438 ],
       [ 0.41496262,  0.41753942, -0.06771705],
       [-0.40912542,  0.4152254 , -0.06780551],
       [ 0.65298176,  0.42654628, -0.067

In [19]:
root = np.nonzero(parents < 0)[0].tolist()
root

[0]

In [20]:
parents

array([-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13,
       14, 16, 17, 18, 19, 20, 21])

In [21]:
a = [1,3,4,5]
a = np.array(a)
np.nonzero(a < 5)

(array([0, 1, 2], dtype=int64),)

In [22]:
root = np.nonzero(parents < 0)[0].tolist()
assert (
    len(root) == 1
), "Body model error. No root joint or multiple root joints: " + str(root)
root = root[0]
depth = [[root]]
visited = set()
visited.add(root)
while len(visited) < len(parents):
    
    print('depth:',depth)
    print('visited:',visited)
    children = sum(
        [
            np.nonzero(parents == parent)[0].tolist()
            for parent in depth[-1]
        ],
        [],
    )
    print('children:',children)
    depth += [children]
    visited.update(children)
    

depth: [[0]]
visited: {0}
children: [1, 2, 3]
depth: [[0], [1, 2, 3]]
visited: {0, 1, 2, 3}
children: [4, 5, 6]
depth: [[0], [1, 2, 3], [4, 5, 6]]
visited: {0, 1, 2, 3, 4, 5, 6}
children: [7, 8, 9]
depth: [[0], [1, 2, 3], [4, 5, 6], [7, 8, 9]]
visited: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
children: [10, 11, 12, 13, 14]
depth: [[0], [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14]]
visited: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
children: [15, 16, 17]
depth: [[0], [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17]]
visited: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
children: [18, 19]
depth: [[0], [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17], [18, 19]]
visited: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
children: [20, 21]
depth: [[0], [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17], [18, 19], [20, 21]]
visited: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 

In [23]:
depth


[[0],
 [1, 2, 3],
 [4, 5, 6],
 [7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17],
 [18, 19],
 [20, 21],
 [22, 23]]

In [25]:
rest_pose.shape

(24, 3)